In [1]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd
import re
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_cols = {
    "பாடல்": [],
    "படம்": [],
    "வருடம்": [],
    "இசையமைப்பாளர்": [],
    "பாடல்வரிகள்": [],
    "பாடகர்கள்": [],
    "பாடலாசிரியர்": []
}
songs_df = pd.DataFrame(df_cols)
songs_df

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்


In [3]:
final_cols = df_cols.keys()
final_cols

dict_keys(['பாடல்', 'படம்', 'வருடம்', 'இசையமைப்பாளர்', 'பாடல் வரிகள்', 'பாடகர்கள்', 'பாடலாசிரியர்'])

In [4]:
origin = 'https://www.tamilbeatslyrics.com/'
url ='https://www.tamilbeatslyrics.com/category/tamil-music-directors/a-r-rahman'
reg = re.compile(r'[a-zA-Z]')

In [5]:
res = req.get(url)
docs = BeautifulSoup(res.text, 'html.parser')

In [6]:
totalPages = int(docs.findAll("a", class_= "page-numbers")[1].string)
totalPages

9

In [7]:
currentPageNumber = 1

while(currentPageNumber <= totalPages):
    currentURL = url + "/page/" + str(currentPageNumber) ## navigate pagination URL
    currentPageResult = req.get(currentURL)
    currentPageDoc = BeautifulSoup(currentPageResult.text, "html.parser")
    headings = currentPageDoc.findAll("a", attrs={"rel" : "bookmark"})


    ### go through the each page of songs lyrics
    for item in headings:
        songURL = (item.attrs)["href"]
        songResult = req.get(songURL) ## navigate to the song details URL
        songDoc = BeautifulSoup(songResult.text, "html.parser") 
        trs = songDoc.find_all("tr")
        
        songDict = {}

        ### travel thorggh the table to get details of songs
        for tr in trs:
            songDict[tr.th.getText(separator=" ").replace(':', '')] = tr.td.getText(separator=" ")

        lyricsParas = songDoc.find_all("p", class_ = "has-text-align-center")

        ### merge lyrics in single sting
        lyrics = ""
        for lyric in lyricsParas:
            if reg.match(lyric.getText(separator=" ")):
                break
            else:
                lyrics += lyric.getText(separator=" ") + " "
        
        songDict['பாடல் வரிகள்'] = lyrics
        
        # Add to the main lyrics frame
        songs_df = songs_df.append(songDict, ignore_index=True)

    print("Page " + str(currentPageNumber) + " completed ...")
    currentPageNumber += 1

songs_df.shape

Page 1 completed ...
Page 2 completed ...
Page 3 completed ...
Page 4 completed ...
Page 5 completed ...
Page 6 completed ...
Page 7 completed ...
Page 8 completed ...
Page 9 completed ...


(92, 12)

In [8]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   பாடல்           47 non-null     object
 1   படம்            47 non-null     object
 2   வருடம்          90 non-null     object
 3   இசையமைப்பாளர்   43 non-null     object
 4   பாடல் வரிகள்    92 non-null     object
 5   பாடகர்கள்       43 non-null     object
 6   பாடலாசிரியர்    43 non-null     object
 7   இசை             47 non-null     object
 8   வரிகள்          47 non-null     object
 9   பாடகர்          47 non-null     object
 10  படத்தின் பெயர்  43 non-null     object
 11  பாடலின் பெயர்   43 non-null     object
dtypes: object(12)
memory usage: 8.8+ KB


In [9]:
songs_df['வருடம்'] = songs_df['வருடம்'].fillna('0')
songs_df.fillna('', inplace = True)

In [10]:
songs_df["பாடல்"] = songs_df[["பாடல்", "பாடலின் பெயர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["படம்"] = songs_df[["படம்", "படத்தின் பெயர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["இசையமைப்பாளர்"] = songs_df[["இசையமைப்பாளர்", "இசை"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["பாடகர்கள்"] = songs_df[["பாடகர்கள்", "பாடகர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["பாடலாசிரியர்"] = songs_df[["பாடலாசிரியர்", "வரிகள்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்,இசை,வரிகள்,பாடகர்,படத்தின் பெயர்,பாடலின் பெயர்
0,ராஜ்யமா இல்லை இமயமா,பாபா,2002,AR ரஹ்மான்,ராஜ்யமா இல்லை இமயமா எங்கிவன் நாளை எங்கிவன் மன்...,ஜெயச்சந்திரன்,வாலி,AR ரஹ்மான்,வாலி,ஜெயச்சந்திரன்,,
1,அஞ்சாதே ஜீவா,ஜோடி,1999,AR ரஹ்மான்,பெண்: அஞ்சாதே ஜீவா நெஞ்சோடு வா வா ஆனந்த பூவே அ...,"சீர்காழி G சிவசிதம்பரம், ஸ்வர்ணலதா",வைரமுத்து,AR ரஹ்மான்,வைரமுத்து,"சீர்காழி G சிவசிதம்பரம், ஸ்வர்ணலதா",,
2,ஒரு பொய்யாவது சொல் கண்ணே,ஜோடி,1999,AR ரஹ்மான்,ஒரு பொய்யாவது சொல் கண்ணே உன் காதல் நான் தான் எ...,ஹாிஹரண்,வைரமுத்து,AR ரஹ்மான்,வைரமுத்து,ஹாிஹரண்,,
3,சோனியா சோனியா,ரட்சகன்,1997,AR ரஹ்மான்,ஆண்1: சோனியா சோனியா சோனியா சோனியா சோனியா சோனிய...,"உதித் நாராயணன், உன்னிகிருஷ்ணன், ஹாிணி",வைரமுத்து,AR ரஹ்மான்,வைரமுத்து,"உதித் நாராயணன், உன்னிகிருஷ்ணன், ஹாிணி",,
4,நெஞ்சே நெஞ்சே மறந்துவிடு,ரட்சகன்,1997,AR ரஹ்மான்,ஆண்: நெஞ்சே நெஞ்சே மறந்துவிடு நினைவினை கடந்துவ...,"KJ யேசுதாஸ், சாதனா சர்கம்",வைரமுத்து,AR ரஹ்மான்,வைரமுத்து,"KJ யேசுதாஸ், சாதனா சர்கம்",,
...,...,...,...,...,...,...,...,...,...,...,...,...
87,ராக்கோழி ரெண்டும்,உழவன்,1993,AR ரஹ்மான்,ஆண்: கோழி ரெண்டு முழிச்சிருக்கு ரெண்டும் கூடாம...,"KJ யேசுதாஸ், ஸ்வர்ணலதா",வாலி,,,,உழவன்,ராக்கோழி ரெண்டும்
88,சரட்டு வண்டியிலே,காற்று வெளியிடை,2017,ஏ.ஆா்.ரஹ்மான்,பெண்: சரட்டு வண்டியிலே சிரட்டொலியில ஓரந் தொிஞ...,"AR ரைஹானா, திப்பு, நிக்கிதா காந்தி",வைரமுத்து,,,,காற்று வெளியிடை,சரட்டு வண்டியிலே
89,அன்பில் அவன்,விண்ணைத்தாண்டி வருவாயா,2010,ஏ.ஆர்.ரஹ்மான்,குழு: அன்பில் அவன் சேர்த்த இதை மனிதரே வெறுக்கா...,"தேவன் ஏகாம்பரம், சின்மயீ",தாமரை,,,,விண்ணைத்தாண்டி வருவாயா,அன்பில் அவன்
90,நீதானே நீதானே,மெர்சல்,2017,ஏ.ஆர்.ரகுமான்,பெண்: நீதானே நீதானே என் நெஞ்சை தட்டும் சத்தம் ...,"ஏ.ஆர்.ரகுமான், ஸ்ரேயா கோஷல்",விவேக்,,,,மெர்சல்,நீதானே நீதானே


In [11]:
import numpy as np

songs_df['வருடம்'] = songs_df["வருடம்"].apply(np.int64)
songs_df.dtypes

பாடல்             object
படம்              object
வருடம்             int64
இசையமைப்பாளர்     object
பாடல் வரிகள்      object
பாடகர்கள்         object
பாடலாசிரியர்      object
இசை               object
வரிகள்            object
பாடகர்            object
படத்தின் பெயர்    object
பாடலின் பெயர்     object
dtype: object

In [12]:
songs_df['வருடம்'] = songs_df["வருடம்"].replace(0, np.NAN)
songs_df[final_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   பாடல்          92 non-null     object 
 1   படம்           92 non-null     object 
 2   வருடம்         90 non-null     float64
 3   இசையமைப்பாளர்  92 non-null     object 
 4   பாடல் வரிகள்   92 non-null     object 
 5   பாடகர்கள்      92 non-null     object 
 6   பாடலாசிரியர்   92 non-null     object 
dtypes: float64(1), object(6)
memory usage: 5.2+ KB


In [13]:
final_df = songs_df[final_cols]
final_df.head(40)

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்
0,ராஜ்யமா இல்லை இமயமா,பாபா,2002.0,AR ரஹ்மான்,ராஜ்யமா இல்லை இமயமா எங்கிவன் நாளை எங்கிவன் மன்...,ஜெயச்சந்திரன்,வாலி
1,அஞ்சாதே ஜீவா,ஜோடி,1999.0,AR ரஹ்மான்,பெண்: அஞ்சாதே ஜீவா நெஞ்சோடு வா வா ஆனந்த பூவே அ...,"சீர்காழி G சிவசிதம்பரம், ஸ்வர்ணலதா",வைரமுத்து
2,ஒரு பொய்யாவது சொல் கண்ணே,ஜோடி,1999.0,AR ரஹ்மான்,ஒரு பொய்யாவது சொல் கண்ணே உன் காதல் நான் தான் எ...,ஹாிஹரண்,வைரமுத்து
3,சோனியா சோனியா,ரட்சகன்,1997.0,AR ரஹ்மான்,ஆண்1: சோனியா சோனியா சோனியா சோனியா சோனியா சோனிய...,"உதித் நாராயணன், உன்னிகிருஷ்ணன், ஹாிணி",வைரமுத்து
4,நெஞ்சே நெஞ்சே மறந்துவிடு,ரட்சகன்,1997.0,AR ரஹ்மான்,ஆண்: நெஞ்சே நெஞ்சே மறந்துவிடு நினைவினை கடந்துவ...,"KJ யேசுதாஸ், சாதனா சர்கம்",வைரமுத்து
5,கண்ணாமூச்சி ஏனடா,கண்டுகொண்டேன் கண்டுகொண்டேன்,2000.0,AR ரஹ்மான்,கண்ணாமூச்சி ஏனடா கண்ணாமூச்சி ஏனடா என் கண்ணா கண...,KS சித்ரா,வைரமுத்து
6,கொஞ்சும் மைனாக்களே,கண்டுகொண்டேன் கண்டுகொண்டேன்,2000.0,AR ரஹ்மான்,கொஞ்சும் மைனாக்களே கொஞ்சும் மைனாக்களே என் குரல...,ஷாதனா சர்கம்,வைரமுத்து
7,அலைகடல் ஆழம்,பொன்னியின் செல்வன் 01,2022.0,AR ரகுமான்,அலைகடல் ஆழம் நிலவு அறியாதோ ஏலோ ஏலேலோ அடி மன தா...,அண்டாரா நண்டி,சிவா ஆனந்த்
8,மயிலிறகே மயிலிறகே,அன்பே ஆருயிரே,2005.0,AR ரஹ்மான்,ஆண்: மயிலிறகே மயிலிறகே வருடுகிறாய் மெல்ல மழை ந...,"நரேஷ் ஐயர், மதுஸ்ரீ",வாலி
9,மச்சான் எப்போ வர போற,வெந்து தணிந்தது காடு,2022.0,AR ரகுமான்,பெண்: ஹேய் மல்லிப்பூ வெச்சு வெச்சு வாடுதே அந்த...,மதுஸ்ரீ,தாமரை


In [14]:
final_df = final_df.replace('', np.nan).dropna().reset_index(drop = True)
final_df['பாடல் வரிகள்'][10]

'ஆண்: உன்ன நெனச்சதும் மனசு மயங்குதே மனசு மயங்கி தான் முத்தம் கேட்டதே ஆண்: முத்தம் கேட்டதும் முகம் சிவந்ததே முகம் சிவந்ததும் இதயம் திறந்ததே பெண்: இதயம் திறந்ததும் ஆசை நுழைந்ததே ஆசை நுழைந்ததும் தூரம் குறைந்ததே ஆண்: உன்ன நெனச்சதும் மனசு மயங்குதே மனசு மயங்கி தான் முத்தம் கேட்டதே பெண்: தூரம் குறைந்ததும் பேச தோணுதே பேச பேச தான் இன்னும் பிடிக்குதே பெண்: பிடிக்கும் என்றதால் நடிக்க தோணுதே நடிக்கும் போதிலே சிரிப்பு வந்ததே ஆண்: சிரிப்பு வந்ததும் நெருக்கம் ஆகுதே நெருங்கி பார்க்கையில் நேசம் புரியுதே பெண்: நேசங்களால் கைகள் இணைந்ததே கை சேர்ந்ததால் கவலை மறந்ததே ஒன்: தோள் சாயவும் தொலைந்து போகவும் கடைசியாக ஓரிடம் கிடைத்ததே ஆண்: உன்ன நெனச்சதும் மனசு மயங்குதே மனசு மயங்கி தான் முத்தம் கேட்டதே ஆண்: மழை வருகிற மணம் வருவது எனக்கு மட்டுமா தனிமையில் அதை முகர்கிற சுகம் உனக்கும் கிட்டுமா பெண்: இரு புறம் அதில் நடுவினில் புயல் எனக்கு மட்டுமா மழையென வரும் மரகத குரல் சுவரில் முட்டுமா ஆண்: எனது புதையல் மணலிலே கொதிக்கும் அனலிலே இருந்தும் விரைவில் கை சேரும் பயணம் முடிவிலே ஆண்: உன்ன நெனச்சதும் மனசு மயங்குதே மனசு மயங்கி த

In [19]:
final_df

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்
0,ராஜ்யமா இல்லை இமயமா,பாபா,2002.0,AR ரஹ்மான்,ராஜ்யமா இல்லை இமயமா எங்கிவன் நாளை எங்கிவன் மன்...,ஜெயச்சந்திரன்,வாலி
1,அஞ்சாதே ஜீவா,ஜோடி,1999.0,AR ரஹ்மான்,பெண்: அஞ்சாதே ஜீவா நெஞ்சோடு வா வா ஆனந்த பூவே அ...,"சீர்காழி G சிவசிதம்பரம், ஸ்வர்ணலதா",வைரமுத்து
2,ஒரு பொய்யாவது சொல் கண்ணே,ஜோடி,1999.0,AR ரஹ்மான்,ஒரு பொய்யாவது சொல் கண்ணே உன் காதல் நான் தான் எ...,ஹாிஹரண்,வைரமுத்து
3,சோனியா சோனியா,ரட்சகன்,1997.0,AR ரஹ்மான்,ஆண்1: சோனியா சோனியா சோனியா சோனியா சோனியா சோனிய...,"உதித் நாராயணன், உன்னிகிருஷ்ணன், ஹாிணி",வைரமுத்து
4,நெஞ்சே நெஞ்சே மறந்துவிடு,ரட்சகன்,1997.0,AR ரஹ்மான்,ஆண்: நெஞ்சே நெஞ்சே மறந்துவிடு நினைவினை கடந்துவ...,"KJ யேசுதாஸ், சாதனா சர்கம்",வைரமுத்து
...,...,...,...,...,...,...,...
85,ராக்கோழி ரெண்டும்,உழவன்,1993.0,AR ரஹ்மான்,ஆண்: கோழி ரெண்டு முழிச்சிருக்கு ரெண்டும் கூடாம...,"KJ யேசுதாஸ், ஸ்வர்ணலதா",வாலி
86,சரட்டு வண்டியிலே,காற்று வெளியிடை,2017.0,ஏ.ஆா்.ரஹ்மான்,பெண்: சரட்டு வண்டியிலே சிரட்டொலியில ஓரந் தொிஞ...,"AR ரைஹானா, திப்பு, நிக்கிதா காந்தி",வைரமுத்து
87,அன்பில் அவன்,விண்ணைத்தாண்டி வருவாயா,2010.0,ஏ.ஆர்.ரஹ்மான்,குழு: அன்பில் அவன் சேர்த்த இதை மனிதரே வெறுக்கா...,"தேவன் ஏகாம்பரம், சின்மயீ",தாமரை
88,நீதானே நீதானே,மெர்சல்,2017.0,ஏ.ஆர்.ரகுமான்,பெண்: நீதானே நீதானே என் நெஞ்சை தட்டும் சத்தம் ...,"ஏ.ஆர்.ரகுமான், ஸ்ரேயா கோஷல்",விவேக்


In [16]:
final_df.to_csv('Tamil A.R Rahman Melody Songs Lyrics_Scrapped.csv', index=False,  encoding='utf-8')
with open('Tamil A.R Rahman Melody Songs Lyrics_Scrapped.json', 'w', encoding='utf-8') as fileJSON:
    final_df.to_json(fileJSON, orient='records', lines=True, force_ascii=False)